## importação dos pacotes


In [1]:
import pyodbc as PDB  # Drive para conexão com banco de dados
import pandas  as pd
import sys
import traceback
from datetime import date


## variaveis de controle

In [2]:
server_target = 'rgprd-sqlsrv-prd01.database.windows.net'
database_master = 'master'
username_server_target = 'implanta.net'
password_target = 'M@st3rP0w3r@zur3Prd'
driver= '{ODBC Driver 17 for SQL Server}'

server_source ='DS10\MSSQLSERVER2017'
database_source ="Elmah"
user_source ="wesley"
password_source ="M@st3rP0w3r@zur3Dev"

dados_df = pd.DataFrame()

data_atual = date.today()



## Utils


In [3]:
def get_conection(driver,server,database,username,password):
    _conexao = PDB.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    return _conexao  

## Motodo para executar o select em cada banco de dados

In [4]:
def executa_sql_get_elmah(database_name):
    dados = pd.DataFrame()
    try:
        _con = get_conection(driver,server_target,database_name,username_server_target,password_target)
        
        query =" EXEC dbo.GetElmahErrosOnAplication"
        

        SQL_exec_procedure = pd.read_sql_query(query,_con)

        dados = pd.DataFrame(SQL_exec_procedure, columns=['ErrorId','Application','Host','Type','Source','Message','User','StatusCode', 
         'TimeUtc','AllXml','ServerName'])
         
        ##dados.set_index('ErrorId',inplace=True)
        _con.close() 
        return dados

    except TypeError:
        _con.close()
    return dados


    

## executa teste

In [5]:
dados =  executa_sql_get_elmah('crea-sp.implanta.net.br')

,ErrorId,Application,Host,Type,Source,Message,User,StatusCode,TimeUtc,AllXml,ServerName
0,666E19D6-C981-4757-A0C3-D15561161B46,Siscont,RGPRD-WSVM-02,Error,,Execution DelayedJobScheduler is in the Failed...,,0,2020-11-28 17:58:05.620,"<error\r\n application=""Hangfire""\r\n host=""...",crea-sp.implanta.net.br
1,E9EBB2E1-1FBC-4EEE-85A3-882B8A52EC7C,Siscont,RGPRD-WSVM-02,Error,System.Data,Execution BackgroundServerProcess is still in ...,,0,2020-11-28 18:11:43.143,"<error\r\n application=""Hangfire""\r\n host=""...",crea-sp.implanta.net.br
2,08658EF6-68DD-4E3A-9EA2-53CA7CA01068,Logon,RGPRD-WSVM-02,System.Data.SqlClient.SqlException,.Net SqlClient Data Provider,Transaction (Process ID 608) was deadlocked on...,,0,2020-11-30 11:13:16.843,"<error\r\n application=""Logon""\r\n host=""RGP...",crea-sp.implanta.net.br
3,13A354F0-ADA2-416F-8AE0-F4D99FE68982,Logon,RGPRD-WSVM-02,System.Transactions.TransactionAbortedException,System.Transactions,A transação foi anulada.,,500,2020-11-30 11:13:16.917,"<error\r\n application=""Logon""\r\n host=""RGP...",crea-sp.implanta.net.br
4,58D9EF1D-4A17-4348-A552-2CB632C06578,Portal Transpaência,RGPRD-WSVM-02,System.Web.HttpException,System.Web.Mvc,A public action method 'BuscarEntityFaleConosc...,,404,2020-12-01 10:42:39.370,"<error\r\n application=""Portal Transpaência""\...",crea-sp.implanta.net.br


In [6]:
dados.count()

ErrorId        1870
Application    1870
Host           1870
Type           1870
Source         1870
Message        1870
User           1870
StatusCode     1870
TimeUtc        1870
AllXml         1870
ServerName     1870
dtype: int64

In [13]:

dados = dados[~dados['Message'].str.startswith("Execution")] # remove os erros do Hangfire

dados = dados[~dados['Message'].str.startswith("Configuração do parcelamento não informada.")] # Remove os erros de migração

dados = dados[~dados['Message'].str.startswith("payment_type")] # Remove as mensagens de cartão de creditos

dados = dados[~dados['Message'].str.contains("order_number")] # Remove as mensagens de cartão de creditos






## Conecta no servidor target e source 

In [15]:
database = 'master'

conexao =get_conection(driver,server_target,database,username_server_target,password_target)

target_conection =get_conection(driver,server_source,database_source,user_source,password_source)
 

## Testa a Conexão com Source

In [16]:
cursor = target_conection.cursor()


cursor.execute("SELECT @@Version") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

cursor.close()

Microsoft SQL Server 2017 (RTM) - 14.0.1000.169 (X64) 
	Aug 22 2017 17:04:49 
	Copyright (C) 2017 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 18362: ) (Hypervisor)



## Monta a query para ler todos so bancos de dados

In [17]:
SQL_Query = pd.read_sql_query(
'''SELECT  D.database_id,D.name FROM  sys.databases AS D
 order by D.database_id''', conexao)

##Monta o DataFrame

In [18]:
databases = pd.DataFrame(SQL_Query, columns=['database_id','name'])

In [19]:
databases.count()

database_id    504
name           504
dtype: int64

## Executa um filtro para executar as querys somente nos databases selecionados

In [20]:
bancos_filtrados = databases[~databases['name'].str.contains("master|Manager|ajuda|Conversor|conversor|Copy|DNE|automationjobs|Configuracao|rglab|ESPELHO|apresentacao|COPY|Homologacao|EXCLUIR|202|PreMig|conversor|treinamento|patrimonio")]

In [21]:
bancos_filtrados.count()

database_id    411
name           411
dtype: int64

In [24]:
bancos_filtrados.sample(10)

,database_id,name
280,292,coren-rs.implanta.net.br
328,341,crf-se.implanta.net.br
107,111,coren-go.implanta.net.br
310,322,crp-se.implanta.net.br
302,314,crp-rs.implanta.net.br
224,236,crfa-mg.implanta.net.br
243,255,crfa-sp.implanta.net.br
189,195,crmv-pa.implanta.net.br
88,92,cau-pi.implanta.net.br
270,282,crf-mg.implanta.net.br


##Executa um for each para cada coluna datetime2

In [25]:
target_conection =get_conection(driver,server_source,database_source,user_source,password_source)

In [29]:
def insertElmah_intoTarget(data,target_conection):
    cursor = target_conection.cursor()
    
    for i,row in data.iterrows():
        sql =""
        sql = ("INSERT INTO Staging.ELMAH_Error(ErrorId,Application,Host,Type,Source,Message,[User],StatusCode,TimeUtc,AllXml,ServerName)"
        "SELECT '{0}', '{1}' ,'{2}','{3}','{4}','{5}','{6}',{7},'{8}','{9}', '{10}' "
        " WHERE NOT EXISTS(SELECT 1 FROM Staging.ELMAH_Error AS EE WHERE EE.ErrorId ='{0}' )").format(row["ErrorId"],
        row["Application"],row["Host"],row["Type"],row['Source'],row["Message"].replace("'","''"),row["User"],
        row["StatusCode"],row["TimeUtc"].strftime("%Y-%m-%d %H:%M:%S"),row["AllXml"].replace("'","''"),row["ServerName"])
        cursor.execute(sql)
         
    # print(sql)
# the connection is not autocommitted by default, so we must commit to save our # changes 
    target_conection.commit()
    #return sql


In [30]:
insertElmah_intoTarget(dados,target_conection)

In [31]:
try:
    database =""
    for i, linha in bancos_filtrados.sort_values("name").iterrows():
        database =linha['name']
        retorno = executa_sql_get_elmah(linha['name'])
        
        # Executa o tratamento dos dados
        retorno = retorno[~retorno['Message'].str.startswith("Execution")] # remove os erros do Hangfire

        retorno = retorno[~retorno['Message'].str.startswith("Configuração do parcelamento não informada.")] # Remove os erros de migração

        retorno = retorno[~retorno['Message'].str.startswith("payment_type")] # Remove as mensagens de cartão de creditos

        retorno = retorno[~retorno['Message'].str.contains("order_number")] # Remove as mensagens de cartão de creditos

        retorno = retorno[~retorno['Message'].str.contains("Monitoramento")] # Remove as mensagens de cartão de creditos

        retorno = retorno[~retorno['Message'].str.contains("amount")] # Remove as mensagens de cartão de creditos

        







        print(database,"Total:",retorno["ErrorId"].count())
        comando = insertElmah_intoTarget(retorno,target_conection)

except BaseException as ex:
    # Get current system exception
    ex_type, ex_value, ex_traceback = sys.exc_info()

    # Extract unformatter stack traces as tuples
    trace_back = traceback.extract_tb(ex_traceback)

    # Format stacktrace
    stack_trace = list()

    for trace in trace_back:
        stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))

    print("Exception type : %s " % ex_type.__name__)
    print("Exception message : %s" %ex_value)
    print("Stack trace : %s" %stack_trace)
    print("database : %s" %database)
    print(linha['name'],tableName,coluna,idregistro)         
            
            
        





caa-ma.implanta.net.br Total: 16
cau-ac.implanta.net.br Total: 61
cau-al.implanta.net.br Total: 163
cau-am.implanta.net.br Total: 110
cau-ap.implanta.net.br Total: 113
cau-ba.implanta.net.br Total: 131
cau-br.implanta.net.br Total: 1486
cau-ce.implanta.net.br Total: 153
cau-df.implanta.net.br Total: 123
cau-es.implanta.net.br Total: 187
cau-go.implanta.net.br Total: 216
cau-ma.implanta.net.br Total: 301
cau-mg.implanta.net.br Total: 232
cau-ms.implanta.net.br Total: 92
cau-mt.implanta.net.br Total: 56
cau-pa.implanta.net.br Total: 102
cau-pb.implanta.net.br Total: 163
cau-pe.implanta.net.br Total: 159
cau-pi.implanta.net.br Total: 113
cau-pr.implanta.net.br Total: 198
cau-rj.implanta.net.br Total: 150
cau-rn.implanta.net.br Total: 128
cau-ro.implanta.net.br Total: 137
cau-rr.implanta.net.br Total: 110
cau-rs.implanta.net.br Total: 152
cau-sc.implanta.net.br Total: 379
cau-se.implanta.net.br Total: 212
cau-sp.implanta.net.br Total: 811
cau-to.implanta.net.br Total: 122
cfa-br.implanta.n

In [ ]:
target_conection.close()
conexao.close()